# 3D Object Detection Evaluation Tutorial

Welcome to the 3D object detection evaluation tutorial! We'll walk through the steps to submit your detections to the competition server.

In [ ]:
from av2.evaluation.detection.eval import evaluate
from av2.evaluation.detection.utils import DetectionCfg
from pathlib import Path
from av2.utils.io import read_feather, read_all_annotations

### Constructing the evaluation configuration

The `DetectionCfg` class stores the configuration for the 3D object detection challenge.

- During evaluation, we remove _all_ cuboids which are not within the region-of-interest (ROI) which spatially is a 5 meter dilation of the drivable area isocontour. 

- **NOTE**: If you would like to _locally_ enable this behavior, you **must** pass in the directory to sensor dataset (to build the raster maps from the included vector maps).

In [ ]:
dataset_dir = (
    Path.home() / "data" / "datasets" / "av2" / "sensor"
)  # Path to your AV2 sensor dataset directory.
competition_cfg = DetectionCfg(
    dataset_dir=dataset_dir
)  # Defaults to competition parameters.

In [ ]:
split = "val"
gts = read_all_annotations(
    dataset_dir=dataset_dir, split=split
)  # Contains all annotations in a particular split.
display(gts)

## Preparing detections for submission.

The evaluation expects the following 14 fields within a `pandas.DataFrame`:

- `tx_m`: x-component of the object translation in the egovehicle reference frame.
- `ty_m`: y-component of the object translation in the egovehicle reference frame.
- `tz_m`: z-component of the object translation in the egovehicle reference frame.
- `length_m`: Object extent along the x-axis in meters.
- `width_m`: Object extent along the y-axis in meters.
- `height_m`: Object extent along the z-axis in meters.
- `qw`: Real quaternion coefficient.
- `qx`: First quaternion coefficient.
- `qy`: Second quaternion coefficient.
- `qz`: Third quaternion coefficient.
- `score`: Object confidence.
- `log_id`: Log id associated with the detection.
- `timestamp_ns`: Timestamp associated with the detection.
- `category`: Object category.

Additional details can be found in [SUBMISSION_FORMAT.md](../src/av2/evaluation/detection/SUBMISSION_FORMAT.md).

In [ ]:
# If you've already aggregated your detections into one file.
dts_path = Path("detections.feather")
dts = read_feather(dts_path)

dts, gts, metrics = evaluate(dts, gts, cfg=competition_cfg)  # Evaluate instances.

In [ ]:
display(metrics)

Finally, if you would like to submit to the evaluation server, you just need to export your detections into a `.feather` file. This can be done by:

```python
dts.to_feather("detections.feather")
```